In [73]:
# !pip install billboard.py
# !pip install SpotifyClientCredentials
# !pip install numpy
# !pip install pandas
# !pip install matplotlib


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import billboard


### Getting the top 100 songs in the last 5 years
using billboard api

In [75]:
def from_list_to_df(array):
  di = {'title' :[],'artist':[]}
  for x in array:
    di['title'].append(x.title)
    di['artist'].append(x.artist)
  return pd.DataFrame(di, columns=list(di.keys()))

In [77]:
chart = list()
for x in range(2022,2017,-1):
  chart.extend(billboard.ChartData('hot-100-songs',year = x))



In [ ]:
df = from_list_to_df(chart)
df['popular'] = 1
len(df['artist'].unique())

325

In [ ]:
df['artist'].unique()

array(['Glass Animals', 'Harry Styles', 'The Kid LAROI & Justin Bieber',
       'Adele', 'Ed Sheeran', 'Jack Harlow', 'Latto', 'Justin Bieber',
       'Kodak Black', 'Elton John & Dua Lipa',
       'Future Featuring Drake & Tems', 'Lizzo', 'Lil Nas X',
       'Imagine Dragons X JID', 'Lil Nas X & Jack Harlow', 'GAYLE',
       'Doja Cat', 'Morgan Wallen', 'Bad Bunny & Chencho Corleone',
       'Bad Bunny', 'Kate Bush',
       'Carolina Gaitan, Mauro Castillo, Adassa, Rhenzy Feliz, Diane Guerrero, Stephanie Beatriz & Encanto Cast',
       'Post Malone Featuring Doja Cat', 'Steve Lacy',
       'Nicky Youre & dazy', 'Post Malone & The Weeknd', 'Olivia Rodrigo',
       'Em Beihold', 'Drake Featuring 21 Savage', 'Cody Johnson',
       'Walker Hayes', 'Luke Combs', 'OneRepublic', 'Beyonce',
       'Zach Bryan', 'The Weeknd & Ariana Grande',
       'Silk Sonic (Bruno Mars & Anderson .Paak)', 'Dua Lipa', 'Lil Baby',
       'Doja Cat & The Weeknd', 'Cole Swindell',
       'Gunna & Future Featuri

### Getting popluar and unpopular songs by each artist
i will use spotify api

In [ ]:
%pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = 'eac0e304f0e44239b90c988baef9d999'
client_secret = ''
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [ ]:
def get_song_names_by_artist(artist_name,data):
    """Gets the song names for a given artist."""
    # Search for songs by artist name.
    songs = []
    # row = {'title' :[],'artist':artist_name, 'popular':0 }
    results = sp.search(q='artist:' + artist_name, type="track",limit = 10)
    # For each song in the search results, get the name and id properties.
    for track in results["tracks"]["items"]:
      if track["name"] not in data["title"].values and  track['album']['release_date'] >='2018-01-01' and '2022-12-31'>= track['album']['release_date'] and track["name"] not in songs:
        songs.append(track["name"])
    return pd.DataFrame({'title' :songs,'artist':artist_name, 'popular':0 })

In [ ]:
for x in df['artist'].unique():
  df = pd.concat([df,get_song_names_by_artist(x,df)], ignore_index=True)

In [ ]:
df

,title,artist,popular
0,Heat Waves,Glass Animals,1
1,As It Was,Harry Styles,1
2,Stay,The Kid LAROI & Justin Bieber,1
3,Easy On Me,Adele,1
4,Shivers,Ed Sheeran,1
...,...,...,...
1223,Lady,Brett Young,0
1224,Catch,Brett Young,0
1225,Here Tonight,Brett Young,0
1226,Chapters,Brett Young,0


In [ ]:
df = (df.drop_duplicates())
df = df.reset_index(drop=True)
df

,title,artist,popular
0,Heat Waves,Glass Animals,1
1,As It Was,Harry Styles,1
2,Stay,The Kid LAROI & Justin Bieber,1
3,Easy On Me,Adele,1
4,Shivers,Ed Sheeran,1
...,...,...,...
1180,Lady,Brett Young,0
1181,Catch,Brett Young,0
1182,Here Tonight,Brett Young,0
1183,Chapters,Brett Young,0


In [ ]:
print(df.nunique())
df.drop(df[df.index > 700].index, inplace=True)
print(df.nunique())


title      1183
artist      325
popular       2
dtype: int64
title      699
artist     325
popular      2
dtype: int64


### Downloading Songs from Youtube DL API

In [ ]:
%pip install yt-dlp

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install ffmpeg-python
%pip install youtube_dl


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [79]:
import yt_dlp
import os

def download_song(song_name, artist_name):
    file_path = 'E:/youssef ashmawy/programming projects/Data Science/Popular and unpopular songs/Music'
    search_query = f'{song_name} {artist_name} audio'
    output_file = os.path.join(file_path, f'{song_name} - {artist_name}')
    
    # Check if the file already exists
    if os.path.exists(output_file + '.mp3'):
        print(f"Skipping download for '{song_name} - {artist_name}'. File already exists.")
        return

    ydl_opts = {
        'default_search': f'ytsearch:{search_query}',
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'postprocessor_args': [
            '-ar', '44100'
        ],
        'prefer_ffmpeg': True,
        'noplaylist': True,
        'nocheckcertificate': True,
        'quiet': True,
        'outtmpl': output_file + '.%(ext)s',
        'audioformat': "mp3",
        'ffmpeg_location': 'E:/youssef ashmawy/programming projects/Data Science/Popular and unpopular songs/ffmpeg-2023-07-10-git-1c61c24f5f-essentials_build/bin' ,
        'external_downloader': 'ffmpeg',
        'external_downloader_args': ['-loglevel', 'quiet'],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([search_query])


In [80]:
for index,name in df.iterrows():
  print(index)
  download_song(df.loc[index,'title'], df.loc[index,'artist'])

0
Skipping download for 'Heat Waves - Glass Animals'. File already exists.
1
Skipping download for 'As It Was - Harry Styles'. File already exists.
2
Skipping download for 'Stay - The Kid LAROI & Justin Bieber'. File already exists.
3
Skipping download for 'Easy On Me - Adele'. File already exists.
4
Skipping download for 'Shivers - Ed Sheeran'. File already exists.
5
Skipping download for 'First Class - Jack Harlow'. File already exists.
6
Skipping download for 'Big Energy - Latto'. File already exists.
7
Skipping download for 'Ghost - Justin Bieber'. File already exists.
8
Skipping download for 'Super Gremlin - Kodak Black'. File already exists.
9
Skipping download for 'Cold Heart (PNAU Remix) - Elton John & Dua Lipa'. File already exists.
10
Skipping download for 'Wait For U - Future Featuring Drake & Tems'. File already exists.
11
Skipping download for 'About Damn Time - Lizzo'. File already exists.
12
Skipping download for 'Bad Habits - Ed Sheeran'. File already exists.
13
Skippin

In [ ]:
#Fixing file names
import os

def rename_files(old_file_name, song_name, artist_name):
    directory = "E:/youssef ashmawy/programming projects/Data Science/Popular and unpopular songs/Music"
    for filename in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, filename)):
            if filename.startswith(old_file_name) and filename.endswith(".mp3"):
                current_name = os.path.join(directory, filename)
                new_name = os.path.join(directory, song_name + ' - ' + artist_name + '.mp3')
                new_name = new_name.replace('<', '').replace('>', '').replace(':', '').replace('"', '').replace('/', '').replace('\\', '').replace('|', '').replace('?', '').replace('*', '')
                os.rename(current_name, new_name)
                return new_name



In [ ]:
for index,name in df.iterrows():
  old = df.loc[index,'title'] + ' - ' + df.loc[index,'title']
  new = rename_files(old,df.loc[index,'title'],df.loc[index,'artist'])
  new_name = df.loc[index,'title'] + ' - ' + df.loc[index,'artist']
  new_name = new_name.replace('<', '').replace('>', '').replace(':', '').replace('"', '').replace('/', '').replace('\\', '').replace('|', '').replace('?', '').replace('*', '')
  df.loc[index,'file_path'] ="E:/youssef ashmawy/programming projects/Data Science/Popular and unpopular songs/Music/" + str(new_name)
  # print(rename_files(old,df.loc[index,'title'],df.loc[index,'artist']))

In [81]:
print(df.loc[75])
print(df.loc[75,'file_path'])

title                          All Too Well (Taylor's Version)
artist                                            Taylor Swift
popular                                                      1
file_path    E:/youssef ashmawy/programming projects/Data S...
Name: 75, dtype: object
E:/youssef ashmawy/programming projects/Data Science/Popular and unpopular songs/Music/All Too Well (Taylor's Version) - Taylor Swift.mp3


In [82]:
df


,title,artist,popular,file_path
0,Heat Waves,Glass Animals,1,E:/youssef ashmawy/programming projects/Data S...
1,As It Was,Harry Styles,1,E:/youssef ashmawy/programming projects/Data S...
2,Stay,The Kid LAROI & Justin Bieber,1,E:/youssef ashmawy/programming projects/Data S...
3,Easy On Me,Adele,1,E:/youssef ashmawy/programming projects/Data S...
4,Shivers,Ed Sheeran,1,E:/youssef ashmawy/programming projects/Data S...
...,...,...,...,...
694,everything i wanted,Billie Eilish,0,E:/youssef ashmawy/programming projects/Data S...
695,bad guy,Billie Eilish,0,E:/youssef ashmawy/programming projects/Data S...
696,WAP (feat. Megan Thee Stallion),Cardi B,0,E:/youssef ashmawy/programming projects/Data S...
697,Bodak Yellow,Cardi B,0,E:/youssef ashmawy/programming projects/Data S...


### Extracting the repeated chorus using pychorus

In [ ]:
for index,name in df.iterrows():
  new_name = df.loc[index,'title'] + ' - ' + df.loc[index,'artist'] + '.mp3'
  new_name = new_name.replace('<', '').replace('>', '').replace(':', '').replace('"', '').replace('/', '').replace('\\', '').replace('|', '').replace('?', '').replace('*', '')
  df.loc[index,'file_path'] ="E:/youssef ashmawy/programming projects/Data Science/Popular and unpopular songs/Music/" + new_name
  df.loc[index,'title'] = df.loc[index,'title'].replace('<', '').replace('>', '').replace(':', '').replace('"', '').replace('/', '').replace('\\', '').replace('|', '').replace('?', '').replace('*', '')


In [ ]:
%pip install pychorus


In [83]:
import pychorus
import soundfile as sf
import os

def extract_chorus(file_path, song_name, artist_name, pop):
    try:
            # Check if the file already exists

        # Extract the repeated chorus
        if pop == 1:
            new_name = str(song_name) + ' - ' + str(artist_name) + ' chorus' + '.mp3'
            new_name = new_name.replace('<', '').replace('>', '').replace(':', '').replace('"', '').replace('/', '').replace('\\', '').replace('|', '').replace('?', '').replace('*', '')
            output_file = 'E:/youssef ashmawy/programming projects/Data Science/Popular and unpopular songs/chorus/popular/' + new_name

        else:
            new_name = str(song_name) + ' - ' + str(artist_name) + ' chorus' + '.mp3'
            new_name = new_name.replace('<', '').replace('>', '').replace(':', '').replace('"', '').replace('/', '').replace('\\', '').replace('|', '').replace('?', '').replace('*', '')
            output_file = 'E:/youssef ashmawy/programming projects/Data Science/Popular and unpopular songs/chorus/unpopular/' + new_name
            if os.path.exists(output_file):
                print(f"Skipping '{song_name} - {artist_name}'. File already exists.")
                return
        pychorus.find_and_output_chorus(file_path, output_file)
        print(f"Chorus extracted for {song_name} - {artist_name}")
        
    except FileNotFoundError:
        print(f"File not found for {song_name} - {artist_name}. Skipping...")

In [ ]:
%pip install numpy==1.24


Note: you may need to restart the kernel to use updated packages.


In [85]:
for index,name in data.iterrows():
    extract_chorus(data.loc[index,'file_path'],data.loc[index,'title'],data.loc[index,'artist'],data.loc[index,'popular'])
    

Best chorus found at 3 min 18.71 sec
Chorus extracted for Heat Waves - Glass Animals
Best chorus found at 1 min 25.77 sec
Chorus extracted for As It Was - Harry Styles
Best chorus found at 1 min 20.74 sec
Chorus extracted for Stay - The Kid LAROI & Justin Bieber
Best chorus found at 2 min 55.04 sec
Chorus extracted for Easy On Me - Adele
Best chorus found at 3 min 4.70 sec
Chorus extracted for Shivers - Ed Sheeran
Best chorus found at 2 min 27.59 sec
Chorus extracted for First Class - Jack Harlow
Best chorus found at 2 min 15.58 sec
Chorus extracted for Big Energy - Latto
Best chorus found at 2 min 37.88 sec
Chorus extracted for Ghost - Justin Bieber
Best chorus found at 2 min 12.96 sec
Chorus extracted for Super Gremlin - Kodak Black
Best chorus found at 1 min 40.61 sec
Chorus extracted for Cold Heart (PNAU Remix) - Elton John & Dua Lipa
Best chorus found at 2 min 27.03 sec
Chorus extracted for Wait For U - Future Featuring Drake & Tems
Best chorus found at 1 min 27.46 sec
Chorus extr

c:\Users\Yousef\AppData\Local\Programs\Python\Python311\Lib\site-packages\pychorus\helpers.py:118: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(input_file)
c:\Users\Yousef\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Best chorus found at 1 min 50.80 sec
Chorus extracted for Fingers Crossed - Lauren Spencer-Smith
Best chorus found at 4 min 37.40 sec
Chorus extracted for All Too Well (Taylor's Version) - Taylor Swift
Best chorus found at 2 min 58.07 sec
Chorus extracted for Party - Bad Bunny & Rauw Alejandro
No choruses were detected. Try a smaller search duration
Chorus extracted for Despues de La Playa - Bad Bunny
Best chorus found at 2 min 41.37 sec
Chorus extracted for You Should Probably Leave - Chris Stapleton
Best chorus found at 1 min 26.65 sec
Chorus extracted for Rockin' Around The Christmas Tree - Brenda Lee
Best chorus found at 2 min 20.50 sec
Chorus extracted for Broadway Girls - Lil Durk Featuring Morgan Wallen
Best chorus found at 1 min 50.86 sec
Chorus extracted for Take My Name - Parmalee
Best chorus found at 0 min 55.30 sec
Chorus extracted for What Happened To Virgil - Lil Durk Featuring Gunna
Best chorus found at 0 min 50.67 sec
Chorus extracted for Puffin On Zootiez - Future
Best

In [7]:
import librosa
from scipy.stats import skew, kurtosis

In [ ]:
features = ['chroma_stft', 'chroma_cqt', 'chroma_cens', 'mfcc', 'spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff', 'tonnetz', 'spectral_contrast']

for index, row in df.iterrows():
    
    new_name = str(row.title) + ' - ' + str(row.artist)
    new_name = new_name.replace('<', '').replace('>', '').replace(':', '').replace('"', '').replace('/', '').replace('\\', '').replace('|', '').replace('?', '').replace('*', '')
    audio_file = f'E:/youssef ashmawy/programming projects/Data Science/Popular and unpopular songs/chorus/all/{new_name} chorus.mp3'
    try:
        y, sr = librosa.load(audio_file)
    except FileNotFoundError:
        print(f"File not found: {audio_file}")
        continue  # Skip to the next iteration if the file is not found
    
    for feature in features:
        feature_array = getattr(librosa.feature, feature)(y=y, sr=sr)
        for index2 in range(0, feature_array.shape[0]):
            df.loc[index, f'{feature}{index2}min'] = np.min(feature_array[index2])
            df.loc[index, f'{feature}{index2}mean'] = np.mean(feature_array[index2])
            df.loc[index, f'{feature}{index2}median'] = np.median(feature_array[index2])
            df.loc[index, f'{feature}{index2}max'] = np.max(feature_array[index2])
            df.loc[index, f'{feature}{index2}std'] = np.std(feature_array[index2])
            df.loc[index, f'{feature}{index2}skew'] = skew(feature_array[index2])
            df.loc[index, f'{feature}{index2}kurtosis'] = kurtosis(feature_array[index2])


In [ ]:
features=['rms','zero_crossing_rate']
for index, row in df.iterrows():
    new_name = str(row.title) + ' - ' + str(row.artist)
    new_name = new_name.replace('<', '').replace('>', '').replace(':', '').replace('"', '').replace('/', '').replace('\\', '').replace('|', '').replace('?', '').replace('*', '')
    audio_file = f'E:/youssef ashmawy/programming projects/Data Science/Popular and unpopular songs/chorus/all/{new_name} chorus.mp3'
    try:
        y, sr = librosa.load(audio_file)
    except FileNotFoundError:
        print(f"File not found: {audio_file}")
        continue  # Skip to the next iteration if the file is not found
    
    for feature in features:
        feature_array = getattr(librosa.feature, feature)(y=y)
        for index2 in range(0, feature_array.shape[0]):
            df.loc[index, f'{feature}{index2}min'] = np.min(feature_array[index2])
            df.loc[index, f'{feature}{index2}mean'] = np.mean(feature_array[index2])
            df.loc[index, f'{feature}{index2}median'] = np.median(feature_array[index2])
            df.loc[index, f'{feature}{index2}max'] = np.max(feature_array[index2])
            df.loc[index, f'{feature}{index2}std'] = np.std(feature_array[index2])
            df.loc[index, f'{feature}{index2}skew'] = skew(feature_array[index2])
            df.loc[index, f'{feature}{index2}kurtosis'] = kurtosis(feature_array[index2])


In [22]:
df.isnull().sum()

title                           0
artist                          0
popular                         0
file_path                       0
chroma_stft0min                31
                               ..
zero_crossing_rate0median      31
zero_crossing_rate0max         31
zero_crossing_rate0std         31
zero_crossing_rate0skew        31
zero_crossing_rate0kurtosis    31
Length: 522, dtype: int64

In [ ]:
df.to_csv('E:/youssef ashmawy/programming projects/Data Science/Popular and unpopular songs/final.csv', index=False)